In [1]:
import sys 
import os 
import json 
from datetime import datetime 
import re 

from snowflake.snowpark.session import Session 
import snowflake.snowpark.functions as F 
import snowflake.snowpark.types as T 

from snowflake.ml.modeling.preprocessing import OneHotEncoder 

In [2]:
connection_params = {
    "connection_name": "default"
}

session = Session.builder.configs(connection_params).create()

session.use_database("ML_SNOWPARK_CI_CD_DB")
session.use_schema("ML_PROCESSING")



In [8]:
def sproc_process_input(session: Session) -> T.Variant:
    # Import Libraries
    from datetime import datetime
    import re

    from snowflake.snowpark.session import Session
    import snowflake.snowpark.functions as F
    from snowflake.snowpark.functions import col

    from snowflake.ml.modeling.preprocessing import OneHotEncoder

    # Creating a Snowpark DataFrame
    application_record_sdf = session.table('ML_SNOWPARK_CI_CD_DB.DATA_PROCESSING.APPLICATION_RECORD_STREAM')
    print('Application table size\t: ',application_record_sdf.count())

    if application_record_sdf.count() == 0:
        print('\nAPPLICATION_RECORD_STREAM is empty')
        sys.exit()

    # Selecting a few columns for modeling
    cols_numerical = ['AMT_INCOME_TOTAL', 'DAYS_EMPLOYED', 'FLAG_MOBIL', 'CNT_FAM_MEMBERS']
    cols_categorical = ['CODE_GENDER', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE']
    application_record_sdf = application_record_sdf[cols_numerical+cols_categorical]

    # Perform One-Hot-Encoding for categorical columns
    my_ohe_encoder = OneHotEncoder(input_cols=cols_categorical, output_cols=cols_categorical, drop_input_cols=True)
    prepared_sdf = my_ohe_encoder.fit(application_record_sdf).transform(application_record_sdf)

    # Cleaning column names to make it easier for future referencing
    cols = prepared_sdf.columns
    for old_col in cols:
        new_col = re.sub(r'[^a-zA-Z0-9_]', '', old_col)
        new_col = new_col.upper()
        prepared_sdf = prepared_sdf.rename(col(old_col), new_col)


    temp_df = session.table('ML_SNOWPARK_CI_CD_DB.ML_PROCESSING.PROCESSED_INPUT').limit(0)
    final_table = temp_df.natural_join(prepared_sdf,
                                        how='outer').fillna(0)
    final_table = final_table.with_column('TIMESTAMP', F.current_timestamp())

    if final_table.count() > 0:
        count = final_table.count()
        print('\nRows to be written to ML_SNOWPARK_CI_CD_DB.ML_PROCESSING.PROCESSED_INPUT = ',final_table.count())
        final_table.write.mode('append').save_as_table("ML_SNOWPARK_CI_CD.ML_PROCESSING.PROCESSED_INPUT")
    else:
        print('Final DF Empty')
        sys.exit()
        
    return str(f'{count} rows written to the ML_PROCESSING.PROCESSED_INPUT stream at ' + str(datetime.now()))


sproc_process_input(session) 

Application table size	:  2


/opt/anaconda3/envs/pysnowpark_ml_ops/lib/python3.9/site-packages/snowflake/snowpark/session.py:2618: UserWarning: Pandas Dataframe has non-standard index of type <class 'pandas.core.indexes.numeric.Int64Index'> which will not be written. Consider changing the index to pd.RangeIndex(start=0,...,step=1) or call reset_index() to keep index as column(s)
  success, _, _, ci_output = write_pandas(
/opt/anaconda3/envs/pysnowpark_ml_ops/lib/python3.9/site-packages/snowflake/snowpark/session.py:2618: UserWarning: Pandas Dataframe has non-standard index of type <class 'pandas.core.indexes.numeric.Int64Index'> which will not be written. Consider changing the index to pd.RangeIndex(start=0,...,step=1) or call reset_index() to keep index as column(s)
  success, _, _, ci_output = write_pandas(


SnowparkSQLException: (1304): 01b7ed2a-0004-0f25-0000-834b00177252: 002003 (42S02): SQL compilation error:
Object 'ML_SNOWPARK_CI_CD_DB.ML_PROCESSING.PROCESSED_INPUT' does not exist or not authorized.

In [4]:
sproc_de = session.sproc.register(func=sproc_process_input,
                                  name='sproc_process_input',
                                  is_permanent=True,
                                  replace=True,
                                  stage_location='@ML_PROCESSING.ML_MODELS',
                                  packages=['snowflake-ml-python',
                                            'snowflake-snowpark-python'
                                           ])

The version of package 'snowflake-ml-python' in the local environment is 1.6.4, which does not fit the criteria for the requirement 'snowflake-ml-python'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'snowflake-snowpark-python' in the local environment is 1.23.0, which does not fit the criteria for the requirement 'snowflake-snowpark-python'. Your UDF might not work when the package version is different between the server and your local environment.
